### Entegra Report Analysis

In [13]:
# pip install geopy

In [14]:
import unicodedata
import datetime as dt
import os
import sys
import re
import numpy as np
import pandas as pd
import math
from pathlib import Path
import nltk
from nltk import word_tokenize 
import shutil
from os.path import isfile, join
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import locale
from locale import atof
from geopy.geocoders import Nominatim
from geopy.point import Point
from geopy.exc import GeocoderTimedOut
from functools import reduce

##### Functions

In [15]:
# custom character alphabet for word
tr = re.compile(r"[abcçdefgğhıijklmnoöprsştuüvyzqxw]+", re.IGNORECASE|re.UNICODE) # Turkish filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n0123456789'
en = re.compile(r"[abcdefghıijklmnopqrstxuvwyz]+", re.IGNORECASE|re.UNICODE) # English
nl = re.compile(r"[abcdefghıijklmnopqrstxuvwyzāăēĕīĭōŏūŭ]+", re.IGNORECASE|re.UNICODE) # Dutch (Flemenk)
fr = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàâæèéêëîïôœùûüÿ]+", re.IGNORECASE|re.UNICODE) # French
de = re.compile(r"[abcdefghıijklmnopqrstxuvwyzäöüß]+", re.IGNORECASE|re.UNICODE) # German
es = re.compile(r"[abcdefghıijklmnopqrstxuvwyzñáéíóú]+", re.IGNORECASE|re.UNICODE) # Spanish (¿¡)
pt = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàáâãéêíóôõú]+", re.IGNORECASE|re.UNICODE) # Portuguese
it = re.compile(r"[abcdefghıijklmnopqrstxuvwyzàéèìòùî]+", re.IGNORECASE|re.UNICODE) # Italian
ar = re.compile(r"[ٿصؼۤڳڲؿڎػڠجڿ٬ٸؽؒؓطۄڀۂؘؔتٚڛےٝڜؖڦ٫ډ۰زۇٖۀ،لۓعٮێڔ۶ؚۧۜڤۏإٞٷؗۖ؈ژۣؕؑٴأۻڸۺگاڴڹۯ؉ْڌ؍ي؟ـٟړۅؐڶُىڽېًۢؠضۚڄٛڏٱۦ٩س٦ڼڂٔۘ٠ښٌٍ۬ٳ۾ٲږذۋٵٜ٘ڞڅںٗهڣۿپڒۥۗڋیؙم؞ثۨٹڵڪظٶۭ١ڭەڨحٕ؎ٺڷٰ۪۫ڻڥۛڑڟټآڡغګ؊ّٯڧڮ؏ۮ؋ؤ٪ؾڗۼق۟دکوِڰڐۃ۽ہفرڇچڝ۴بۈٽڕۡھةٓڃئ؛ڬٙڙڢڱۊَۆۉځ۠ۍۑۙڊنءڈٻشڍ؇۵كخ\ا]+", re.IGNORECASE|re.UNICODE) # Arabic

In [16]:
def convert_one_character_letter(text):
    '''This function converts two byte occupy of letter to one byte unicode character without any visual change \n
    like as Turkish character ç,ş,ö,ğ. \n
    convert_one_character_letter(text): text is any string word or sentence.
    '''
    new_text = unicodedata.normalize('NFC', f"{text}")
    return new_text

In [17]:
def clean_text(text, custom_alp=tr): # for only string in specific language
    '''This function extract custom_alp character from string \n
    clean_text(text, custom_alp): text is any string word or sentence and custom_alp is specific alphabet.
    '''
    # text_clean = re.findall(custom_alp, text)
    text_result_list = []
    text_list = re.findall(r'\S+', text)
    for text_var in text_list:
        text_clean = re.findall(custom_alp, str(text_var))
        text_result_var = "".join(text_clean)
        text_result_list.append(text_result_var)
    text_result = " ".join(text_result_list)
    return text_result

In [18]:
def lower_func(text):
    '''This function convert string character to lowercase. \n
    lower_func(text=str): text is any string word or sentence
    '''
    string_lower = str.lower(text)
    return string_lower

In [19]:
def upper_func(text):
    '''This function convert string character to uppercase. \n
    upper_func(text=str): text is any string word or sentence
    '''
    string_upper = str.upper(text)
    return string_upper

In [20]:
def capitalize_func(text):
    '''This function convert string character to capitalize case. \n
    capitalize_func(text=str): text is any string word or sentence
    '''
    string_capitalize = str.capitalize(text)
    return string_capitalize

In [21]:
def title_func(text):
    '''This function convert string character to title as each word start with capital letter. \n
    title_func(text=str): text is any string word or sentence
    '''
    string_title = str.title(text)
    return string_title

In [22]:
def character_clean_lower_text(text, custom_alp=tr): # for only string in specific language
    '''This function converts two byte occupy of letter to one byte unicode character without any visual change \n
    like as Turkish character ç,ş,ö,ğ. And also extract custom_alp character from string and convert to lowercase. \n
    character_clean_lower_text(text, custom_alp): text is any string word or sentence and custom_alp is specific alphabet.  
    '''
    new_text = unicodedata.normalize('NFC', f"{text}")
    text_result_list = []
    text_list = re.findall(r'\S+', new_text)  # like as word tokenize
    for text_var in text_list:
        text_clean = re.findall(custom_alp, str(text_var))
        text_result_var = "".join(text_clean)
        text_result_list.append(text_result_var)
    text_join = " ".join(text_result_list)
    text_result = str.lower(text_join)
    return text_result

In [23]:
def convert_to_float(value):
    '''This function convert string numeric value to float type. Like as 1.200,15
    convert_to_float(value): value is a string numeric value
    '''
    value = value.replace(".","").replace(",",".")
    return float(value)

In [24]:
def whitespace_del(text):
    '''This function provides removing left right white space.\n
    whitespace_del(text): text is a string value as word or sentence.
    '''
    text_var = str(text)
    text_var = text_var.strip()  # rstrip lstrip
    return text_var

#### Report

In [25]:
input_path = r"C:\Users\user\Desktop\Data Analysis\Adoba\Sales_Report\Data\Entegra\Sales"
output_path = r"C:\Users\user\Downloads"

data_file = r"Entegra_Sales_List (01.01.2024-11.07.2024)"

In [96]:
# df_entegra = pd.read_excel(fr"{input_path}\{data_file}.xlsx")

# df_entegra.to_csv(fr"{output_path}\{data_file}.csv", encoding='utf8', index=False)

In [26]:
df_entegra = pd.read_csv(fr"{input_path}\{data_file}.csv", low_memory=False)
df_entegra

,id,order_number,platform_reference_no,invoice_number,datetime,date_add,entegration,supplier,status,status_name,...,pov_productCode,payment_type,total_product,PazaryerindenGelenOdemeTutar,total_product_quantity,delivery_method,fatura_tarihi,invoice_url,invoice_type,FaturaTipiAdi
0,178856,2280224850,9352213652,NaN,2024-07-11 16:21:44,2024-07-11 16:23:02,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,NaN,NaN,NaN,0,Seciniz
1,178857,2280223780,9352211863,NaN,2024-07-11 16:20:54,2024-07-11 16:23:17,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,Ertesi Gün Kargoda,NaN,NaN,0,Seciniz
2,178858,2280223782,9352210132,NaN,2024-07-11 16:20:05,2024-07-11 16:23:19,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,Ertesi Gün Kargoda,NaN,NaN,0,Seciniz
3,178859,2280222421,9352209819,NaN,2024-07-11 16:19:57,2024-07-11 16:23:22,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,2,Ertesi Gün Kargoda,NaN,NaN,0,Seciniz
4,178860,2280220893,9352209340,NaN,2024-07-11 16:19:42,2024-07-11 16:23:24,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,NaN,NaN,NaN,0,Seciniz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95054,83601,207423367318-1,207423367318,NaN,2024-01-01 00:18:00,2024-01-01 00:24:34,N11,n11,1,Yeni Siparis,...,160620230178,N11,1,0,1,NaN,2024-01-02 10:46:02,NaN,0,Seciniz
95055,83598,1935764998,2424519644,NaN,2024-01-01 00:14:43,2024-01-01 00:18:56,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,3,Ertesi Gün Kargoda,2024-01-02 06:41:47,NaN,0,Seciniz
95056,83596,1935755636,8061482665,NaN,2024-01-01 00:02:03,2024-01-01 00:04:53,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,Ertesi Gün Kargoda,2024-01-02 08:31:11,NaN,0,Seciniz
95057,83597,201757367316-1,201757367316,NaN,2024-01-01 00:02:00,2024-01-01 00:07:04,N11,n11,1,Yeni Siparis,...,160620230178,N11,1,0,1,NaN,2024-01-02 10:46:02,NaN,0,Seciniz


In [27]:
df_entegra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95059 entries, 0 to 95058
Data columns (total 59 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            95059 non-null  int64  
 1   order_number                  95059 non-null  object 
 2   platform_reference_no         95059 non-null  object 
 3   invoice_number                0 non-null      float64
 4   datetime                      95059 non-null  object 
 5   date_add                      95059 non-null  object 
 6   entegration                   95059 non-null  object 
 7   supplier                      95059 non-null  object 
 8   status                        95059 non-null  int64  
 9   status_name                   95059 non-null  object 
 10  company                       95058 non-null  object 
 11  cargo_code                    92937 non-null  object 
 12  invoice_country               2849 non-null   object 
 13  i

In [28]:
df_entegra.columns = [x.lower() for x in df_entegra.columns]
df_entegra.columns = [x.strip() for x in df_entegra.columns]
df_entegra

,id,order_number,platform_reference_no,invoice_number,datetime,date_add,entegration,supplier,status,status_name,...,pov_productcode,payment_type,total_product,pazaryerindengelenodemetutar,total_product_quantity,delivery_method,fatura_tarihi,invoice_url,invoice_type,faturatipiadi
0,178856,2280224850,9352213652,NaN,2024-07-11 16:21:44,2024-07-11 16:23:02,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,NaN,NaN,NaN,0,Seciniz
1,178857,2280223780,9352211863,NaN,2024-07-11 16:20:54,2024-07-11 16:23:17,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,Ertesi Gün Kargoda,NaN,NaN,0,Seciniz
2,178858,2280223782,9352210132,NaN,2024-07-11 16:20:05,2024-07-11 16:23:19,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,Ertesi Gün Kargoda,NaN,NaN,0,Seciniz
3,178859,2280222421,9352209819,NaN,2024-07-11 16:19:57,2024-07-11 16:23:22,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,2,Ertesi Gün Kargoda,NaN,NaN,0,Seciniz
4,178860,2280220893,9352209340,NaN,2024-07-11 16:19:42,2024-07-11 16:23:24,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,NaN,NaN,NaN,0,Seciniz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95054,83601,207423367318-1,207423367318,NaN,2024-01-01 00:18:00,2024-01-01 00:24:34,N11,n11,1,Yeni Siparis,...,160620230178,N11,1,0,1,NaN,2024-01-02 10:46:02,NaN,0,Seciniz
95055,83598,1935764998,2424519644,NaN,2024-01-01 00:14:43,2024-01-01 00:18:56,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,3,Ertesi Gün Kargoda,2024-01-02 06:41:47,NaN,0,Seciniz
95056,83596,1935755636,8061482665,NaN,2024-01-01 00:02:03,2024-01-01 00:04:53,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,Ertesi Gün Kargoda,2024-01-02 08:31:11,NaN,0,Seciniz
95057,83597,201757367316-1,201757367316,NaN,2024-01-01 00:02:00,2024-01-01 00:07:04,N11,n11,1,Yeni Siparis,...,160620230178,N11,1,0,1,NaN,2024-01-02 10:46:02,NaN,0,Seciniz


In [29]:
df_entegra.columns

Index(['id', 'order_number', 'platform_reference_no', 'invoice_number',
       'datetime', 'date_add', 'entegration', 'supplier', 'status',
       'status_name', 'company', 'cargo_code', 'invoice_country',
       'invoice_city', 'tax_office', 'tax_number', 'tc_id', 'total', 'tax',
       'grand_total', 'sync', 'erp_order_number', 'erp_message',
       'store_order_status', 'store_order_status_name', 'order_process',
       'note', 'supplier1', 'supplier_id', 'cargo_sync', 'cargo_company',
       'cargo_code2', 'cargo_follow_url', 'cargo_finally_statu',
       'cargo_error_message', 'einvoice_error_message', 'invoice_print_name',
       'ship_print_name', 'sevk_tarihi', 'ship_country', 'customer_code',
       'cargo_fee_type_name', 'cargo_packet_type', 'cargo_packet_quantity',
       'grand_total_desi', 'model', 'product_name', 'store_product_name',
       'invoice_name', 'pov_productcode', 'payment_type', 'total_product',
       'pazaryerindengelenodemetutar', 'total_product_quantity',

In [30]:
df_entegra.model.nunique()

1044

In [102]:
df_entegra.pov_productcode.nunique()

386

In [103]:
df_entegra_select_var = df_entegra[["model","pov_productcode","product_name","datetime","total_product_quantity","entegration",\
    "invoice_city","invoice_country","total","tax","grand_total","company","cargo_company","status_name"]]
df_entegra_select_var

,model,pov_productcode,product_name,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,company,cargo_company,status_name
0,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:21:44,1,trendyol,Yalova,NaN,383.33,76.67,460.0,sevim velioglu,trendyolekspress,Onaylandı
1,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:20:54,1,trendyol,Bolu,NaN,154.17,30.83,185.0,Aslı Ünen,trendyolekspress,Onaylandı
2,003954,NaN,AWOX Barmix Çubuk Blender 1000 W,2024-07-11 16:20:05,1,trendyol,Ağrı,NaN,298.33,59.67,358.0,Rahime Alpaslan,trendyolekspress,Onaylandı
3,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:19:57,2,trendyol,İstanbul,NaN,300.00,60.00,360.0,sıla erol,trendyolekspress,Onaylandı
4,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:19:42,1,trendyol,Samsun,NaN,383.33,76.67,460.0,Gülser Doğmuş,trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95054,002583,160620230178,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01 00:18:00,1,N11,Kütahya,NaN,153.64,15.36,169.0,MELTEM KOCA,mng,Yeni Siparis
95055,002979,NaN,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,2024-01-01 00:14:43,3,trendyol,Adıyaman,NaN,318.18,31.82,350.0,Sıla Dikel,trendyolekspress,Onaylandı
95056,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-01-01 00:02:03,1,trendyol,Aydın,NaN,312.50,62.50,375.0,özlem yıldız,trendyolekspress,Onaylandı
95057,002583,160620230178,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01 00:02:00,1,N11,Kocaeli,NaN,153.64,15.36,169.0,ekrem çelik,mng,Yeni Siparis


In [104]:
df_entegra_select_var["invoice_country"].unique()

array([nan, 'Suudi Arabistan', 'Azerbaycan', 'Birleşik Arap Emirlikleri',
       'Türkiye', 'Bahreyn'], dtype=object)

In [105]:
df_entegra_select_var["cargo_company"].unique()

array(['trendyolekspress', 'mng', 'hepsijet', nan, 'eptt', 'yurtici',
       'kolaygelsin', 'surat', 'horoz'], dtype=object)

In [106]:
df_entegra_select_var.loc[:,"invoice_country"] = df_entegra_select_var.loc[:,"invoice_country"].fillna("Türkiye")
df_entegra_select_var

,model,pov_productcode,product_name,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,company,cargo_company,status_name
0,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:21:44,1,trendyol,Yalova,Türkiye,383.33,76.67,460.0,sevim velioglu,trendyolekspress,Onaylandı
1,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:20:54,1,trendyol,Bolu,Türkiye,154.17,30.83,185.0,Aslı Ünen,trendyolekspress,Onaylandı
2,003954,NaN,AWOX Barmix Çubuk Blender 1000 W,2024-07-11 16:20:05,1,trendyol,Ağrı,Türkiye,298.33,59.67,358.0,Rahime Alpaslan,trendyolekspress,Onaylandı
3,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:19:57,2,trendyol,İstanbul,Türkiye,300.00,60.00,360.0,sıla erol,trendyolekspress,Onaylandı
4,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:19:42,1,trendyol,Samsun,Türkiye,383.33,76.67,460.0,Gülser Doğmuş,trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95054,002583,160620230178,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01 00:18:00,1,N11,Kütahya,Türkiye,153.64,15.36,169.0,MELTEM KOCA,mng,Yeni Siparis
95055,002979,NaN,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,2024-01-01 00:14:43,3,trendyol,Adıyaman,Türkiye,318.18,31.82,350.0,Sıla Dikel,trendyolekspress,Onaylandı
95056,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-01-01 00:02:03,1,trendyol,Aydın,Türkiye,312.50,62.50,375.0,özlem yıldız,trendyolekspress,Onaylandı
95057,002583,160620230178,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01 00:02:00,1,N11,Kocaeli,Türkiye,153.64,15.36,169.0,ekrem çelik,mng,Yeni Siparis


In [107]:
df_entegra_select_var.loc[:,"cargo_company"] = df_entegra_select_var.loc[:,"cargo_company"].fillna("diger")
df_entegra_select_var

,model,pov_productcode,product_name,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,company,cargo_company,status_name
0,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:21:44,1,trendyol,Yalova,Türkiye,383.33,76.67,460.0,sevim velioglu,trendyolekspress,Onaylandı
1,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:20:54,1,trendyol,Bolu,Türkiye,154.17,30.83,185.0,Aslı Ünen,trendyolekspress,Onaylandı
2,003954,NaN,AWOX Barmix Çubuk Blender 1000 W,2024-07-11 16:20:05,1,trendyol,Ağrı,Türkiye,298.33,59.67,358.0,Rahime Alpaslan,trendyolekspress,Onaylandı
3,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:19:57,2,trendyol,İstanbul,Türkiye,300.00,60.00,360.0,sıla erol,trendyolekspress,Onaylandı
4,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:19:42,1,trendyol,Samsun,Türkiye,383.33,76.67,460.0,Gülser Doğmuş,trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95054,002583,160620230178,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01 00:18:00,1,N11,Kütahya,Türkiye,153.64,15.36,169.0,MELTEM KOCA,mng,Yeni Siparis
95055,002979,NaN,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,2024-01-01 00:14:43,3,trendyol,Adıyaman,Türkiye,318.18,31.82,350.0,Sıla Dikel,trendyolekspress,Onaylandı
95056,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-01-01 00:02:03,1,trendyol,Aydın,Türkiye,312.50,62.50,375.0,özlem yıldız,trendyolekspress,Onaylandı
95057,002583,160620230178,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01 00:02:00,1,N11,Kocaeli,Türkiye,153.64,15.36,169.0,ekrem çelik,mng,Yeni Siparis


In [108]:
df_entegra_select_duplicated = df_entegra_select_var[df_entegra_select_var.duplicated(keep=False)]
df_entegra_select_duplicated

,model,pov_productcode,product_name,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,company,cargo_company,status_name
4005,001603,NaN,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",2024-07-03 23:01:51,3,HepsiBurada,İstanbul,Türkiye,316.6583,63.3317,379.99,Fatma Yağcı,hepsijet,Onaylandı
4006,001603,NaN,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",2024-07-03 23:01:51,3,HepsiBurada,İstanbul,Türkiye,316.6583,63.3317,379.99,Fatma Yağcı,hepsijet,Onaylandı
4007,001603,NaN,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",2024-07-03 23:01:51,3,HepsiBurada,İstanbul,Türkiye,316.6583,63.3317,379.99,Fatma Yağcı,hepsijet,Onaylandı
4048,002263,AYK8683372407015ÇKMK,"İksa Devrilmez Ayaklı Portatif Kamp Ocağı, Pik...",2024-07-03 21:25:21,1,HepsiBurada,Mardin,Türkiye,383.3250,76.6650,459.99,Fatma AYHAN,hepsijet,Onaylandı
4049,002263,AYK8683372407015ÇKMK,"İksa Devrilmez Ayaklı Portatif Kamp Ocağı, Pik...",2024-07-03 21:25:21,1,HepsiBurada,Mardin,Türkiye,383.3250,76.6650,459.99,Fatma AYHAN,eptt,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93284,001372,NaN,Hobby Life 15'li Yumurta Saklama Kabı,2024-01-05 21:11:20,2,HepsiBurada,Yozgat,Türkiye,141.0750,28.2150,169.29,Şeref YILDIZ,hepsijet,Onaylandı
93909,002202,NaN,Bay-Bayan Wc Yönlendirme Yazısı Kahverengi Ahş...,2024-01-04 10:56:20,2,HepsiBurada,Sakarya,Türkiye,183.3167,36.6633,219.98,İhsan EFİLOĞLU,hepsijet,Onaylandı
93910,002202,NaN,Bay-Bayan Wc Yönlendirme Yazısı Kahverengi Ahş...,2024-01-04 10:56:20,2,HepsiBurada,Sakarya,Türkiye,183.3167,36.6633,219.98,İhsan EFİLOĞLU,hepsijet,Onaylandı
94613,002839,NaN,Yeni Nesil Mor Kedi Kumu Küreği Mor,2024-01-02 14:58:22,1,HepsiBurada,Antalya,Türkiye,116.6583,23.3317,139.99,Gül Dereli,hepsijet,Onaylandı


In [109]:
# df_entegra_select_duplicated.to_excel(fr"{output_path}\Entegra Aynı Sipariş Tekrarlama.xlsx", index=False)

In [110]:
# pd.to_datetime(df_entegra_select_var['datetime'], format='%Y-%m-%d %H:%M:%S')

In [111]:
df_entegra_select_var = df_entegra_select_var.drop_duplicates()
df_entegra_select_var = df_entegra_select_var.reset_index(drop=True)
df_entegra_select_var

,model,pov_productcode,product_name,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,company,cargo_company,status_name
0,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:21:44,1,trendyol,Yalova,Türkiye,383.33,76.67,460.0,sevim velioglu,trendyolekspress,Onaylandı
1,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:20:54,1,trendyol,Bolu,Türkiye,154.17,30.83,185.0,Aslı Ünen,trendyolekspress,Onaylandı
2,003954,NaN,AWOX Barmix Çubuk Blender 1000 W,2024-07-11 16:20:05,1,trendyol,Ağrı,Türkiye,298.33,59.67,358.0,Rahime Alpaslan,trendyolekspress,Onaylandı
3,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:19:57,2,trendyol,İstanbul,Türkiye,300.00,60.00,360.0,sıla erol,trendyolekspress,Onaylandı
4,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:19:42,1,trendyol,Samsun,Türkiye,383.33,76.67,460.0,Gülser Doğmuş,trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94833,002583,160620230178,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01 00:18:00,1,N11,Kütahya,Türkiye,153.64,15.36,169.0,MELTEM KOCA,mng,Yeni Siparis
94834,002979,NaN,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,2024-01-01 00:14:43,3,trendyol,Adıyaman,Türkiye,318.18,31.82,350.0,Sıla Dikel,trendyolekspress,Onaylandı
94835,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-01-01 00:02:03,1,trendyol,Aydın,Türkiye,312.50,62.50,375.0,özlem yıldız,trendyolekspress,Onaylandı
94836,002583,160620230178,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01 00:02:00,1,N11,Kocaeli,Türkiye,153.64,15.36,169.0,ekrem çelik,mng,Yeni Siparis


In [112]:
df_entegra_select_var.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94838 entries, 0 to 94837
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   model                   94837 non-null  object 
 1   pov_productcode         25291 non-null  object 
 2   product_name            94837 non-null  object 
 3   datetime                94838 non-null  object 
 4   total_product_quantity  94838 non-null  int64  
 5   entegration             94838 non-null  object 
 6   invoice_city            94837 non-null  object 
 7   invoice_country         94838 non-null  object 
 8   total                   94838 non-null  float64
 9   tax                     94838 non-null  float64
 10  grand_total             94838 non-null  float64
 11  company                 94837 non-null  object 
 12  cargo_company           94838 non-null  object 
 13  status_name             94838 non-null  object 
dtypes: float64(3), int64(1), object(10)
me

In [113]:
df_entegra_select_var.loc[:,'datetime'] = pd.to_datetime(df_entegra_select_var.loc[:,'datetime'], format='%Y-%m-%d %H:%M:%S')
df_entegra_select_var.head(5)

,model,pov_productcode,product_name,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,company,cargo_company,status_name
0,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:21:44,1,trendyol,Yalova,Türkiye,383.33,76.67,460.0,sevim velioglu,trendyolekspress,Onaylandı
1,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:20:54,1,trendyol,Bolu,Türkiye,154.17,30.83,185.0,Aslı Ünen,trendyolekspress,Onaylandı
2,003954,NaN,AWOX Barmix Çubuk Blender 1000 W,2024-07-11 16:20:05,1,trendyol,Ağrı,Türkiye,298.33,59.67,358.0,Rahime Alpaslan,trendyolekspress,Onaylandı
3,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:19:57,2,trendyol,İstanbul,Türkiye,300.00,60.00,360.0,sıla erol,trendyolekspress,Onaylandı
4,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:19:42,1,trendyol,Samsun,Türkiye,383.33,76.67,460.0,Gülser Doğmuş,trendyolekspress,Onaylandı


In [114]:
pd.to_datetime(df_entegra_select_var["datetime"]).dt.date.head(5)

0    2024-07-11
1    2024-07-11
2    2024-07-11
3    2024-07-11
4    2024-07-11
Name: datetime, dtype: object

In [115]:
pd.to_datetime(df_entegra_select_var["datetime"]).dt.time.head(5)

0    16:21:44
1    16:20:54
2    16:20:05
3    16:19:57
4    16:19:42
Name: datetime, dtype: object

In [116]:
df_entegra_select_var.insert(3,"date",pd.to_datetime(df_entegra_select_var["datetime"]).dt.date)
df_entegra_select_var.insert(4,"time",pd.to_datetime(df_entegra_select_var["datetime"]).dt.time)
df_entegra_select_var.head(5)

,model,pov_productcode,product_name,date,time,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,company,cargo_company,status_name
0,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11,16:21:44,2024-07-11 16:21:44,1,trendyol,Yalova,Türkiye,383.33,76.67,460.0,sevim velioglu,trendyolekspress,Onaylandı
1,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11,16:20:54,2024-07-11 16:20:54,1,trendyol,Bolu,Türkiye,154.17,30.83,185.0,Aslı Ünen,trendyolekspress,Onaylandı
2,003954,NaN,AWOX Barmix Çubuk Blender 1000 W,2024-07-11,16:20:05,2024-07-11 16:20:05,1,trendyol,Ağrı,Türkiye,298.33,59.67,358.0,Rahime Alpaslan,trendyolekspress,Onaylandı
3,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11,16:19:57,2024-07-11 16:19:57,2,trendyol,İstanbul,Türkiye,300.00,60.00,360.0,sıla erol,trendyolekspress,Onaylandı
4,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11,16:19:42,2024-07-11 16:19:42,1,trendyol,Samsun,Türkiye,383.33,76.67,460.0,Gülser Doğmuş,trendyolekspress,Onaylandı


In [117]:
df_entegra_select_var.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94838 entries, 0 to 94837
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   model                   94837 non-null  object 
 1   pov_productcode         25291 non-null  object 
 2   product_name            94837 non-null  object 
 3   date                    94838 non-null  object 
 4   time                    94838 non-null  object 
 5   datetime                94838 non-null  object 
 6   total_product_quantity  94838 non-null  int64  
 7   entegration             94838 non-null  object 
 8   invoice_city            94837 non-null  object 
 9   invoice_country         94838 non-null  object 
 10  total                   94838 non-null  float64
 11  tax                     94838 non-null  float64
 12  grand_total             94838 non-null  float64
 13  company                 94837 non-null  object 
 14  cargo_company           94838 non-null

In [118]:
object_columns = df_entegra_select_var.select_dtypes(include='object')
object_columns

,model,pov_productcode,product_name,date,time,datetime,entegration,invoice_city,invoice_country,company,cargo_company,status_name
0,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11,16:21:44,2024-07-11 16:21:44,trendyol,Yalova,Türkiye,sevim velioglu,trendyolekspress,Onaylandı
1,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11,16:20:54,2024-07-11 16:20:54,trendyol,Bolu,Türkiye,Aslı Ünen,trendyolekspress,Onaylandı
2,003954,NaN,AWOX Barmix Çubuk Blender 1000 W,2024-07-11,16:20:05,2024-07-11 16:20:05,trendyol,Ağrı,Türkiye,Rahime Alpaslan,trendyolekspress,Onaylandı
3,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11,16:19:57,2024-07-11 16:19:57,trendyol,İstanbul,Türkiye,sıla erol,trendyolekspress,Onaylandı
4,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11,16:19:42,2024-07-11 16:19:42,trendyol,Samsun,Türkiye,Gülser Doğmuş,trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...
94833,002583,160620230178,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01,00:18:00,2024-01-01 00:18:00,N11,Kütahya,Türkiye,MELTEM KOCA,mng,Yeni Siparis
94834,002979,NaN,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,2024-01-01,00:14:43,2024-01-01 00:14:43,trendyol,Adıyaman,Türkiye,Sıla Dikel,trendyolekspress,Onaylandı
94835,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-01-01,00:02:03,2024-01-01 00:02:03,trendyol,Aydın,Türkiye,özlem yıldız,trendyolekspress,Onaylandı
94836,002583,160620230178,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01,00:02:00,2024-01-01 00:02:00,N11,Kocaeli,Türkiye,ekrem çelik,mng,Yeni Siparis


In [119]:
object_columns.columns

Index(['model', 'pov_productcode', 'product_name', 'date', 'time', 'datetime',
       'entegration', 'invoice_city', 'invoice_country', 'company',
       'cargo_company', 'status_name'],
      dtype='object')

In [120]:
for column in object_columns.columns:
    df_entegra_select_var.loc[:,f"{column}"] = df_entegra_select_var.loc[:,f"{column}"].apply(lambda x: convert_one_character_letter(x))
    df_entegra_select_var.loc[:,f"{column}"] = df_entegra_select_var.loc[:,f"{column}"].apply(lambda x: whitespace_del(x))
    df_entegra_select_var.loc[:,f"{column}"] = df_entegra_select_var.loc[:,f"{column}"].apply(lambda x: title_func(x))
    
df_entegra_select_var.head(10)

,model,pov_productcode,product_name,date,time,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,company,cargo_company,status_name
0,003362,Nan,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11,16:21:44,2024-07-11 16:21:44,1,Trendyol,Yalova,Türkiye,383.33,76.67,460.00,Sevim Velioglu,Trendyolekspress,Onaylandı
1,001602,Nan,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11,16:20:54,2024-07-11 16:20:54,1,Trendyol,Bolu,Türkiye,154.17,30.83,185.00,Aslı Ünen,Trendyolekspress,Onaylandı
2,003954,Nan,Awox Barmix Çubuk Blender 1000 W,2024-07-11,16:20:05,2024-07-11 16:20:05,1,Trendyol,Ağrı,Türkiye,298.33,59.67,358.00,Rahime Alpaslan,Trendyolekspress,Onaylandı
3,001602,Nan,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11,16:19:57,2024-07-11 16:19:57,2,Trendyol,İstanbul,Türkiye,300.00,60.00,360.00,Sıla Erol,Trendyolekspress,Onaylandı
4,003362,Nan,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11,16:19:42,2024-07-11 16:19:42,1,Trendyol,Samsun,Türkiye,383.33,76.67,460.00,Gülser Doğmuş,Trendyolekspress,Onaylandı
5,004286,Nan,Awox Orbit Gizli Rezistanslı Su Isıtıcı Kettle...,2024-07-11,16:18:35,2024-07-11 16:18:35,1,Trendyol,Ankara,Türkiye,366.67,73.33,440.00,Fırat Aksel,Trendyolekspress,Onaylandı
6,001602,Nan,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11,16:17:52,2024-07-11 16:17:52,1,Trendyol,Eskişehir,Türkiye,154.17,30.83,185.00,Derya Ersöz,Trendyolekspress,Onaylandı
7,003362,Nan,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11,16:13:44,2024-07-11 16:13:44,1,Trendyol,Buraidah,Suudi Arabistan,460.00,0.00,460.00,Layla Muhammad,Trendyolekspress,Onaylandı
8,002656,Nan,Sprey Başlıklı Amber Cam 10 Ml Boş - Yedek Şişe,2024-07-11,16:11:43,2024-07-11 16:11:43,1,Trendyol,Sivas,Türkiye,20.83,4.17,24.99,Buse Nur Kaya,Mng,Onaylandı
9,003362,Nan,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11,16:09:44,2024-07-11 16:09:44,1,Trendyol,İstanbul,Türkiye,383.33,76.67,460.00,Canan Aydın,Trendyolekspress,Onaylandı


In [121]:
# df_entegra_select_var[object_columns.columns] = object_columns.apply(lambda x: x.str.lower())
# df_entegra_select_var

In [122]:
# selected_column = ["product_name","entegration","invoice_city","invoice_country","cargo_company","cargo_company","status_name"] 
# for column in selected_column:
#     df_entegra_select_var[f"{column}"] = df_entegra_select_var[f"{column}"].apply(lambda x: whitespace_del(x))
#     
# df_entegra_select_var.head(10)

In [123]:
# selected_column2 = ["entegration","invoice_city","invoice_country","cargo_company","cargo_company","status_name"] 
# for column in selected_column2:
#     df_entegra_select_var[f"{column}"] = df_entegra_select_var[f"{column}"].apply(lambda x: capitalize_func(x))
# 
# df_entegra_select_var.head(10)

##### Temp

In [124]:
df_entegra_select_var

,model,pov_productcode,product_name,date,time,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,company,cargo_company,status_name
0,003362,Nan,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11,16:21:44,2024-07-11 16:21:44,1,Trendyol,Yalova,Türkiye,383.33,76.67,460.0,Sevim Velioglu,Trendyolekspress,Onaylandı
1,001602,Nan,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11,16:20:54,2024-07-11 16:20:54,1,Trendyol,Bolu,Türkiye,154.17,30.83,185.0,Aslı Ünen,Trendyolekspress,Onaylandı
2,003954,Nan,Awox Barmix Çubuk Blender 1000 W,2024-07-11,16:20:05,2024-07-11 16:20:05,1,Trendyol,Ağrı,Türkiye,298.33,59.67,358.0,Rahime Alpaslan,Trendyolekspress,Onaylandı
3,001602,Nan,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11,16:19:57,2024-07-11 16:19:57,2,Trendyol,İstanbul,Türkiye,300.00,60.00,360.0,Sıla Erol,Trendyolekspress,Onaylandı
4,003362,Nan,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11,16:19:42,2024-07-11 16:19:42,1,Trendyol,Samsun,Türkiye,383.33,76.67,460.0,Gülser Doğmuş,Trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94833,002583,160620230178,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01,00:18:00,2024-01-01 00:18:00,1,N11,Kütahya,Türkiye,153.64,15.36,169.0,Meltem Koca,Mng,Yeni Siparis
94834,002979,Nan,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,2024-01-01,00:14:43,2024-01-01 00:14:43,3,Trendyol,Adıyaman,Türkiye,318.18,31.82,350.0,Sıla Dikel,Trendyolekspress,Onaylandı
94835,003362,Nan,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-01-01,00:02:03,2024-01-01 00:02:03,1,Trendyol,Aydın,Türkiye,312.50,62.50,375.0,Özlem Yıldız,Trendyolekspress,Onaylandı
94836,002583,160620230178,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01,00:02:00,2024-01-01 00:02:00,1,N11,Kocaeli,Türkiye,153.64,15.36,169.0,Ekrem Çelik,Mng,Yeni Siparis


In [125]:
df_mikro_price = pd.read_excel(r"C:\Users\user\Downloads\Kopya Ürün Listesi Fiyat Bilgisi.xlsx")  # ????
df_mikro_price = df_mikro_price.rename(columns={"Ürün Adı":"product_name","Model Kodu":"model","Gelişi":"geliş_fiyat","KDV ORANI":"KDV Oranı"})
df_mikro_price

,product_name,Varyant,Marka,Renk,model,Stok Kodu,SKU,Barkod,Gelişi,KDV Oranı
0,"%100 Doğal At Kılı Selülit Fırçası, Doğal At K...",NaN,proclis,NaN,002972,002972,NaN,NaN,60,20.0
1,"%100 Doğal Keçi Kılı Yüz Bakım Fırçası, Yüz Ba...",NaN,proclis,Ahşap,002975,002975,NaN,NaN,25,20.0
2,%100 Doğal Sarı Kantaron Yağı 50 ml,NaN,proclis,NaN,002960,002960,NaN,NaN,28.74,20.0
3,0-3 Yaş Zeka Kartları Dokun Hisset Tanı Eğitim...,NaN,Circle Toys,NaN,002932,002932,NaN,NaN,118,20.0
4,1 Adet Ayakkabı Hurcu Gri Çizgili,NaN,ALAS,Gri,002924,002924,NaN,NaN,156,10.0
...,...,...,...,...,...,...,...,...,...,...
1573,Şeffaf Kedi Ve Köpek Düz Sırt Taşıma Çantası Sarı,NaN,GOBYPET,Sarı,003865,003865,NaN,NaN,200,20.0
1574,Şeffaf Kedi ve Köpek Düz Sırt Taşıma Çantası 8 KG,NaN,NaN,NaN,003865,003865,NaN,NaN,200,20.0
1575,Şeffaf Tekerlekli Kedi Köpek Taşıma Valizi Çek...,NaN,Bagline,ŞEFFAF,BAG-21,f,NaN,NaN,NaN,NaN
1576,Şişme Tabanlı 3 Boğumlu Sunset Havuz 114x25 cm,NaN,Intex,Çok Renkli,004004,004004,NaN,NaN,145,20.0


In [126]:
mikro_model_set = set(df_mikro_price["model"])

In [127]:
mikro_model_list = list(mikro_model_set)

In [128]:
df_entegra_select_var[df_entegra_select_var["model"].isin(mikro_model_list)]

,model,pov_productcode,product_name,date,time,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,company,cargo_company,status_name
0,003362,Nan,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11,16:21:44,2024-07-11 16:21:44,1,Trendyol,Yalova,Türkiye,383.33,76.67,460.0,Sevim Velioglu,Trendyolekspress,Onaylandı
1,001602,Nan,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11,16:20:54,2024-07-11 16:20:54,1,Trendyol,Bolu,Türkiye,154.17,30.83,185.0,Aslı Ünen,Trendyolekspress,Onaylandı
2,003954,Nan,Awox Barmix Çubuk Blender 1000 W,2024-07-11,16:20:05,2024-07-11 16:20:05,1,Trendyol,Ağrı,Türkiye,298.33,59.67,358.0,Rahime Alpaslan,Trendyolekspress,Onaylandı
3,001602,Nan,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11,16:19:57,2024-07-11 16:19:57,2,Trendyol,İstanbul,Türkiye,300.00,60.00,360.0,Sıla Erol,Trendyolekspress,Onaylandı
4,003362,Nan,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11,16:19:42,2024-07-11 16:19:42,1,Trendyol,Samsun,Türkiye,383.33,76.67,460.0,Gülser Doğmuş,Trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94833,002583,160620230178,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01,00:18:00,2024-01-01 00:18:00,1,N11,Kütahya,Türkiye,153.64,15.36,169.0,Meltem Koca,Mng,Yeni Siparis
94834,002979,Nan,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,2024-01-01,00:14:43,2024-01-01 00:14:43,3,Trendyol,Adıyaman,Türkiye,318.18,31.82,350.0,Sıla Dikel,Trendyolekspress,Onaylandı
94835,003362,Nan,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-01-01,00:02:03,2024-01-01 00:02:03,1,Trendyol,Aydın,Türkiye,312.50,62.50,375.0,Özlem Yıldız,Trendyolekspress,Onaylandı
94836,002583,160620230178,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01,00:02:00,2024-01-01 00:02:00,1,N11,Kocaeli,Türkiye,153.64,15.36,169.0,Ekrem Çelik,Mng,Yeni Siparis


In [129]:
df_model_price_no = df_entegra_select_var[~df_entegra_select_var["model"].isin(mikro_model_list)]
df_model_price_no

,model,pov_productcode,product_name,date,time,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,company,cargo_company,status_name
5,004286,Nan,Awox Orbit Gizli Rezistanslı Su Isıtıcı Kettle...,2024-07-11,16:18:35,2024-07-11 16:18:35,1,Trendyol,Ankara,Türkiye,366.67,73.33,440.00,Fırat Aksel,Trendyolekspress,Onaylandı
16,002263,Ayk8683372407029Çkmk,"İksa Devrilmez Ayaklı Portatif Kamp Ocağı, Pik...",2024-07-11,15:46:27,2024-07-11 15:46:27,1,Trendyol,İstanbul,Türkiye,280.00,56.00,336.00,Rabia Acar,Mng,Onaylandı
19,002263,Ayk8683372407029Çkmk,"İksa Devrilmez Ayaklı Portatif Kamp Ocağı, Pik...",2024-07-11,15:38:38,2024-07-11 15:38:38,2,Trendyol,İstanbul,Türkiye,370.90,74.18,445.08,Aylin Uygun,Mng,Onaylandı
25,002263,Ayk8683372407029Çkmk,"İksa Devrilmez Ayaklı Portatif Kamp Ocağı, Pik...",2024-07-11,15:23:45,2024-07-11 15:23:45,1,Trendyol,İstanbul,Türkiye,288.33,57.67,346.00,Mustafa Tercan,Mng,Onaylandı
28,002263,Ayk8683372407029Çkmk,"İksa Devrilmez Ayaklı Portatif Kamp Ocağı, Pik...",2024-07-11,15:17:06,2024-07-11 15:17:06,1,Trendyol,İstanbul,Türkiye,288.33,57.67,346.00,Emre Yıldız,Mng,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94816,Tanımsız.,Nan,Tanımsız,2024-01-01,01:38:13,2024-01-01 01:38:13,1,Trendyol,Konya,Türkiye,677.50,135.50,813.00,Şahide Yeşilmen,Trendyolekspress,Onaylandı
94820,003265,Nan,Vipaş Hostel Kalın Borulu Otel Tipi Çamaşır Ku...,2024-01-01,01:22:17,2024-01-01 01:22:17,1,Trendyol,İstanbul,Türkiye,454.17,90.83,545.00,Sadi̇ye Eri̇mli̇ Dal,Trendyolekspress,Onaylandı
94823,Tanımsız.,Nan,Tanımsız,2024-01-01,01:16:37,2024-01-01 01:16:37,1,Trendyol,Mardin,Türkiye,137.50,27.50,165.00,Tolga Göktürk Fidan,Trendyolekspress,Onaylandı
94825,002062,Nan,Müller Benzi̇n Enjektör Temi̇zleyi̇ci̇ (Üstün ...,2024-01-01,01:03:00,2024-01-01 01:03:00,1,N11,Sakarya,Türkiye,109.78,21.96,131.74,Önder Irme,Mng,Yeni Siparis


In [132]:
df_model_price_no_drop = df_model_price_no.drop_duplicates(subset=['model','pov_productcode'])
df_model_price_no_drop

,model,pov_productcode,product_name,date,time,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,company,cargo_company,status_name
5,004286,Nan,Awox Orbit Gizli Rezistanslı Su Isıtıcı Kettle...,2024-07-11,16:18:35,2024-07-11 16:18:35,1,Trendyol,Ankara,Türkiye,366.6700,73.3300,440.00,Fırat Aksel,Trendyolekspress,Onaylandı
16,002263,Ayk8683372407029Çkmk,"İksa Devrilmez Ayaklı Portatif Kamp Ocağı, Pik...",2024-07-11,15:46:27,2024-07-11 15:46:27,1,Trendyol,İstanbul,Türkiye,280.0000,56.0000,336.00,Rabia Acar,Mng,Onaylandı
52,002263,Ayk8683372407015Çkmk,"İksa Devrilmez Ayaklı Portatif Kamp Ocağı, Pik...",2024-07-11,14:26:32,2024-07-11 14:26:32,1,Trendyol,Afyonkarahisar,Türkiye,304.1700,60.8300,365.00,Hasan Kürşad Yavuz,Mng,Onaylandı
135,001284,Nan,Alas Çok Amaçlı Dörtlü Çekmece Düzenleyici Org...,2024-07-11,11:51:52,2024-07-11 11:51:52,2,Trendyol,Adana,Türkiye,192.4200,38.4800,230.90,Hürü Kaplan,Trendyolekspress,Onaylandı
137,002998,Nan,Doğal El Yapımı Ardıç Katranlı Sabun Çiftli (M...,2024-07-11,11:50:03,2024-07-11 11:50:03,1,Trendyol,Ankara,Türkiye,99.9900,20.0000,119.99,Ömer Faruk Özkan,Trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94296,002374,Nan,Da69 Ahşap Resim Şovalesi 37 Cm,2024-01-02,20:08:22,2024-01-02 20:08:22,1,Trendyol,Ankara,Türkiye,99.9900,20.0000,119.99,Emek Elektri̇k End. A.Ş.,Trendyolekspress,Onaylandı
94600,002061,Nan,Müller Di̇zel Enjektör Temizleyi̇ci̇ (Üstün Al...,2024-01-01,21:45:00,2024-01-01 21:45:00,4,N11,Kayseri,Türkiye,422.8100,84.5600,507.37,Hakan Üzel,Mng,Yeni Siparis
94616,002792,Nan,21512-21010 Karter Tapasi Kisa Universal,2024-01-01,20:50:18,2024-01-01 20:50:18,1,Hepsiburada,İzmir,Türkiye,112.4917,22.4983,134.99,Kadir Inanç Polat,Hepsijet,Onaylandı
94729,002300,Ocakyakıtı25042052,"Osmanlı Ocak Yakıtı, Reşo Yakıt, Alfa Fire Man...",2024-01-01,15:16:06,2024-01-01 15:16:06,1,Hepsiburada,İstanbul,Türkiye,122.8750,24.5750,147.45,Mustafa Bora İmamoğlu,Hepsijet,Onaylandı


In [ ]:
df_model_price_no_drop.to_excel(fr"{output_path}\Entegra Ürün Fiyat Listesinde Olmayan.xlsx", index=False)

##### Entegra Products

In [ ]:
df_entegra_select_product = df_entegra_select_var[["model","pov_productcode","product_name"]]
df_entegra_select_product

In [ ]:
df_entegra_select_product["model"].nunique()

In [ ]:
df_entegra_select_product["pov_productcode"].nunique()

In [ ]:
df_entegra_select_product["product_name"].nunique()

In [ ]:
df_entegra_select_product[df_entegra_select_product.duplicated()]

In [ ]:
df_entegra_select_product = df_entegra_select_product.drop_duplicates()
df_entegra_select_product

In [ ]:
# df_entegra_select_product.to_excel(fr"{output_path}\Entegra Model Product Name.xlsx", index=False)  # Adoba..Data Folder

##### Geo Location

In [ ]:
def reverse_geocoding(lat, lon):
    '''This function is used for giving city or country name from longitude latitude value \n
    reverse_geocoding(lat, lon): lat is geographical lattitude and lon is geographical longitude \n
    ex. \n
    df['address'] = np.vectorize(reverse_geocoding)(df['Latitude'], df['Longitude'])'''
    try:
        location = geolocator.reverse(Point(lat, lon))
        return location.raw['display_name']
    except:
        return None

In [ ]:
def findGeocode(city):
    '''This function is used for giving latitude and longitude values from city or country name \n
    findGeocode(city): city is a city or country \n
    ex. \n
    loc = findGeocode(i) \n 
    lat = loc.latitude \n
    lon = loc.longitude''' 
       
    # try and catch is used to overcome 
    # the exception thrown by geolocator 
    # using geocodertimedout   
    try: 
          
        # Specify the user_agent as your 
        # app name it should not be none 
        geolocator = Nominatim(user_agent="tesst") 
          
        return geolocator.geocode(city) 
      
    except GeocoderTimedOut: 
          
        return findGeocode(city)     

In [ ]:
df_entegra_select_var["invoice_city"].nunique()

In [ ]:
# df_entegra_select_var["invoice_city"].unique()

In [ ]:
df_entegra_select_var[df_entegra_select_var["invoice_city"] == "Nan"]

In [ ]:
nan_index = df_entegra_select_var[df_entegra_select_var["invoice_city"] == "Nan"].index[0]
nan_index

In [ ]:
# df_nan_var = df_entegra_select_var[df_entegra_select_var["invoice_city"] == "Nan"]
# df_nan_var

In [ ]:
# df_nan_var.loc[nan_index,"invoice_city"] = "İstanbul"

In [ ]:
df_entegra_city_select = df_entegra_select_var[["invoice_city", "invoice_country"]]
df_entegra_city_select = df_entegra_city_select.drop_duplicates(subset="invoice_city")
df_entegra_city_select = df_entegra_city_select.reset_index(drop=True)
df_entegra_city_select.head(5)

In [ ]:
# geolocator = Nominatim(user_agent="test")
# 
# # each value from city column 
# # will be fetched and sent to 
# # function find_geocode  
# city = [] 
# latitude = [] 
# longitude = []
# 
# for i in (df_entegra_city_select["invoice_city"]): 
#       
#     if findGeocode(i) != None: 
#            
#         loc = findGeocode(i) 
#           
#         # coordinates returned from  
#         # function is stored into 
#         # two separate list 
#         city.append(i)
#         latitude.append(loc.latitude) 
#         longitude.append(loc.longitude) 
#        
#     # if coordinate for a city not 
#     # found, insert "NaN" indicating  
#     # missing value  
#     else:
#         city.append(np.nan) 
#         latitude.append(np.nan) 
#         longitude.append(np.nan)

In [ ]:
# df_entegra_city_select["latitude"] = latitude
# df_entegra_city_select["longitude"] = longitude 
# df_entegra_city_select

In [ ]:
# df_entegra_city_select.to_excel(fr"{output_path}\Entegra City Location.xlsx", index=False)  # Adoba..Data Folder

##### Entegra Select And Geo Location

In [ ]:
df_city_location = pd.read_excel(r"C:\Users\user\Desktop\Data Analysis\Adoba\Sales_Report\Data\Geo Location\Entegra City Location.xlsx")
df_city_location.drop("id", axis=1, inplace=True)
df_city_location

In [ ]:
# df_city_location[df_city_location["city"].duplicated(keep=False)]

In [ ]:
df_city_location["city"].nunique()

In [ ]:
df_entegra_select_var

In [ ]:
df_entegra_city_location = pd.merge(df_entegra_select_var, df_city_location, how="left", on="invoice_city")
df_entegra_city_location = df_entegra_city_location.drop_duplicates().reset_index(drop=True)
df_entegra_city_location

In [ ]:
df_entegra_select = df_entegra_city_location.drop(["invoice_city","invoice_country"], axis=1)
df_entegra_select

In [ ]:
df_entegra_select["latitude"] = df_entegra_select["latitude"].apply(lambda x: round(x, 6))
df_entegra_select["longitude"] = df_entegra_select["longitude"].apply(lambda x: round(x, 6))
df_entegra_select.head(5)

In [ ]:
print(df_entegra_select["product_name"].dtype)

In [ ]:
print(df_entegra_select["latitude"].dtype)

In [ ]:
df_entegra_select["city"].nunique()

In [ ]:
# df_entegra_select.to_excel(fr"{output_path}\Entegra Sales City Location For Tableau (01.01.2024-11.07.2024).xlsx", index=False)  # Adoba..Data Folder

##### Geo Pandas

In [ ]:
# pip install geopandas
# pip install geodatasets
# pip install cartopy
# pip install mplleaflet

In [ ]:
import pandas as pd
import geopandas
import matplotlib.pyplot as plt
from geodatasets import get_path
from cartopy import crs as ccrs

In [ ]:
path = get_path("naturalearth.land")  # naturalearth plot 
df = geopandas.read_file(path)

df.plot()

In [ ]:
df_geo = df_entegra_select[["country","city","latitude","longitude"]]
df_geo.head(5)

In [ ]:
print(f"{round(df_geo['longitude'].min())}, {round(df_geo['latitude'].min())}, {round(df_geo['longitude'].max())}, {round(df_geo['latitude'].max())}")

In [ ]:
print(f"{round(df_geo['longitude'].min()-5)}, {round(df_geo['latitude'].min()-5)}, {round(df_geo['longitude'].max()+5)}, {round(df_geo['latitude'].max()+5)}")  # for word.clip([,,,])

In [ ]:
gdf = geopandas.GeoDataFrame(
    df_geo, geometry=geopandas.points_from_xy(df_geo.longitude, df_geo.latitude), crs="EPSG:4326"
)
gdf.head(5)

In [ ]:
world = geopandas.read_file(get_path("naturalearth.land"))

# We restrict to South America.
# ax = world.clip([-90, -55, -25, 15]).plot(color="white", edgecolor="black")

# we resrict to sales area
# ax = world.plot(color="white", edgecolor="black")
ax = world.clip([5, 12, 63, 47]).plot(color="white", edgecolor="black")  # clip = f"{round(df_geo['longitude'].min()-5)}, {round(df_geo['latitude'].min()-5)}, 
                                                                         #            {round(df_geo['longitude'].max()+5)}, {round(df_geo['latitude'].max()+5)}"

# We can now plot our ``GeoDataFrame``.
gdf.plot(ax=ax, color="orange")

plt.show()

##### Sales Order Quantity

In [ ]:
df_entegra_select["status_name"].unique()

In [ ]:
entegration_list_all = list(df_entegra_select["entegration"].unique())
entegration_list_all

In [ ]:
df_entegra_select_sales = df_entegra_select[(df_entegra_select["status_name"] == "Onaylandı") | (df_entegra_select["status_name"] == "Yeni Siparis")]
df_entegra_select_sales.reset_index(drop=True, inplace=True)
df_entegra_select_sales

In [ ]:
entegration_list_sales = list(df_entegra_select_sales["entegration"].unique())
entegration_list_sales

In [ ]:
df_sales_quantity_entegration = pd.pivot_table(index="model",columns="entegration", values="total_product_quantity", aggfunc='sum', data=df_entegra_select_sales)
df_sales_quantity_entegration = df_sales_quantity_entegration.fillna(0)
df_sales_quantity_entegration.reset_index(inplace=True)
df_sales_quantity_entegration

In [ ]:
df_sales_quantity_entegration = df_sales_quantity_entegration.rename_axis(None, axis=1)  # del index name (entegration)
df_sales_quantity_entegration

In [ ]:
for entegration in entegration_list_all:
    if entegration not in entegration_list_sales:
        df_sales_quantity_entegration[f"{entegration}"] = 0
    else:
        pass
    
df_sales_quantity_entegration

In [ ]:
# 1 way
df_sales_quantity_entegration["toplam_satış_adet"] = 0
for entg_column in entegration_list_all:
    df_sales_quantity_entegration["toplam_satış_adet"] += df_sales_quantity_entegration[f"{entg_column}"]
# 2 way
# df_sales_quantity_entegration["total_product_sales"] = df_sales_quantity_entegration[entegration_list_all].sum(axis=1)

# 3 way manuel sum technique
# df_sales_quantity_entegration["total_product_sales"] = df_sales_quantity_entegration["Amazon"] + df_sales_quantity_entegration["Ciceksepeti"] + \
#     df_sales_quantity_entegration["Hepsiburada"] + df_sales_quantity_entegration["N11"] + df_sales_quantity_entegration["Trendyol"] + df_sales_quantity_entegration["Woocommerce"]
# df_sales_quantity_entegration

df_sales_quantity_entegration 

In [ ]:
df_sales_name_quantity_entegration  = pd.merge(df_sales_quantity_entegration, df_entegra_select_product, how="left", on="model")
df_sales_name_quantity_entegration.drop_duplicates(inplace=True)
df_sales_name_quantity_entegration

In [ ]:
cols = list(df_sales_name_quantity_entegration.columns)
cols = [cols[-1]] + cols[:-1]
df_sales_name_quantity_entegration = df_sales_name_quantity_entegration[cols]
df_sales_name_quantity_entegration

In [ ]:
df_sales_name_quantity_entegration = df_sales_name_quantity_entegration.sort_values(by="toplam_satış_adet", ascending=False)
df_sales_name_quantity_entegration.reset_index(drop=True, inplace=True)
df_sales_name_quantity_entegration

In [ ]:
df_sales_name_quantity_entegration_drop = df_sales_name_quantity_entegration.drop_duplicates(subset="model")
df_sales_name_quantity_entegration_drop.reset_index(drop=True, inplace=True)
df_sales_name_quantity_entegration_drop

In [ ]:
df_sales_name_quantity_entegration_drop.to_excel(fr"{output_path}\Entegra Ürün Satış Rakamları Adet.xlsx", index=False)

##### Sales Order TL

In [ ]:
df_entegra_select_sales

In [ ]:
df_entegration_model_sales = df_entegra_select_sales.groupby(["model","entegration"])[["total","tax","grand_total"]].sum()
df_entegration_model_sales = df_entegration_model_sales.reset_index()
df_entegration_model_sales.sort_values(by=["entegration","total"], ascending=False, inplace=True)
df_entegration_model_sales.reset_index(drop=True, inplace=True)
df_entegration_model_sales

###### Total

In [ ]:
df_sales_entegration_total = pd.pivot_table(index="model",columns="entegration", values="total", aggfunc='sum', data=df_entegration_model_sales)
df_sales_entegration_total = df_sales_entegration_total.fillna(0)
df_sales_entegration_total.reset_index(inplace=True)
df_sales_entegration_total = df_sales_entegration_total.rename_axis(None, axis=1)
df_sales_entegration_total

In [ ]:
# df_sales_entegration_total.isnull()

In [ ]:
for entegration in entegration_list_all:
    if entegration not in entegration_list_sales:
        df_sales_entegration_total[f"{entegration}"] = 0
    else:
        pass
    
df_sales_entegration_total

In [ ]:
# 1 way
df_sales_entegration_total["toplam_net_satış_tl"] = 0
for entg_column in entegration_list_all:
    df_sales_entegration_total["toplam_net_satış_tl"] += df_sales_entegration_total[f"{entg_column}"]

df_sales_entegration_total 

In [ ]:
df_sales_entegration_total = df_sales_entegration_total.sort_values(by="toplam_net_satış_tl", ascending=False)
df_sales_entegration_total.reset_index(drop=True, inplace=True)
df_sales_entegration_total = df_sales_entegration_total[["model","toplam_net_satış_tl"]]
df_sales_entegration_total

###### Tax

In [ ]:
df_sales_entegration_tax = pd.pivot_table(index="model",columns="entegration", values="tax", aggfunc='sum', data=df_entegration_model_sales)
df_sales_entegration_tax = df_sales_entegration_tax.fillna(0)
df_sales_entegration_tax.reset_index(inplace=True)
df_sales_entegration_tax = df_sales_entegration_tax.rename_axis(None, axis=1)
df_sales_entegration_tax

In [ ]:
for entegration in entegration_list_all:
    if entegration not in entegration_list_sales:
        df_sales_entegration_tax[f"{entegration}"] = 0
    else:
        pass
    
df_sales_entegration_tax

In [ ]:
# 1 way
df_sales_entegration_tax["toplam_vergi_tl"] = 0
for entg_column in entegration_list_all:
    df_sales_entegration_tax["toplam_vergi_tl"] += df_sales_entegration_tax[f"{entg_column}"]

df_sales_entegration_tax

In [ ]:
df_sales_entegration_tax = df_sales_entegration_tax.sort_values(by="toplam_vergi_tl", ascending=False)
df_sales_entegration_tax.reset_index(drop=True, inplace=True)
df_sales_entegration_tax = df_sales_entegration_tax[["model","toplam_vergi_tl"]]
df_sales_entegration_tax

###### Grand Total

In [ ]:
df_sales_entegration_grand_total = pd.pivot_table(index="model",columns="entegration", values="grand_total", aggfunc='sum', data=df_entegration_model_sales)
df_sales_entegration_grand_total = df_sales_entegration_grand_total.fillna(0)
df_sales_entegration_grand_total.reset_index(inplace=True)
df_sales_entegration_grand_total = df_sales_entegration_grand_total.rename_axis(None, axis=1)
df_sales_entegration_grand_total

In [ ]:
for entegration in entegration_list_all:
    if entegration not in entegration_list_sales:
        df_sales_entegration_grand_total[f"{entegration}"] = 0
    else:
        pass
    
df_sales_entegration_grand_total

In [ ]:
# 1 way
df_sales_entegration_grand_total["toplam_brüt_satış_tl"] = 0
for entg_column in entegration_list_all:
    df_sales_entegration_grand_total["toplam_brüt_satış_tl"] += df_sales_entegration_grand_total[f"{entg_column}"]

df_sales_entegration_grand_total

In [ ]:
df_sales_entegration_grand_total = df_sales_entegration_grand_total.sort_values(by="toplam_brüt_satış_tl", ascending=False)
df_sales_entegration_grand_total.reset_index(drop=True, inplace=True)
df_sales_entegration_grand_total = df_sales_entegration_grand_total[["model","toplam_brüt_satış_tl"]]
df_sales_entegration_grand_total

In [ ]:
data_sales_frames = [df_sales_entegration_total, df_sales_entegration_tax, df_sales_entegration_grand_total]

In [ ]:
df_sales_merged = reduce(lambda  left,right: pd.merge(left,right,on=['model'], how='inner'), data_sales_frames)
df_sales_merged = df_sales_merged.drop_duplicates()
df_sales_merged = df_sales_merged.reset_index(drop=True)
df_sales_merged

In [ ]:
df_quantity_sales_entegration_merge  = pd.merge(df_sales_name_quantity_entegration_drop, df_sales_merged, how="left", on="model")
df_quantity_sales_entegration_merge.drop_duplicates(inplace=True)
df_quantity_sales_entegration_merge

In [ ]:
df_quantity_sales_entegration_merge.to_excel(fr"{output_path}\Entegra Ürün Satış Adet Ve TL.xlsx", index=False)

###### Sales Duplicated

In [ ]:
df_sales_dublicated_entegration = df_sales_name_quantity_entegration[df_sales_name_quantity_entegration.duplicated(subset="model", keep=False)]
df_sales_dublicated_entegration

In [ ]:
df_sales_dublicated_entegration.to_excel(fr"{output_path}\Entegra Ürün Satış Rakamları Adet Yenile.xlsx", index=False)

##### Sales Visualization

In [ ]:
df_sales_visual = df_sales_name_quantity_entegration_drop.head(50)
df_sales_visual

In [ ]:
select_sales_visual_list = list(df_sales_visual["model"])
# select_sales_visual_list

In [ ]:
# df_sales_name_quantity_entegration_drop.drop(columns="model", axis=1)

In [ ]:
#dfm = pd.melt(frame=df_sales_name_quantity_entegration_drop, id_vars=["product_name","model"],var_name='cols', value_name='vals')
dfm = df_sales_name_quantity_entegration_drop.melt(id_vars=["product_name","model"],var_name='cols', value_name='vals')
dfm

In [ ]:
dfm_select = dfm[dfm["model"].isin(select_sales_visual_list)]
dfm_select

In [ ]:
plt.figure(figsize=(16,9),dpi=100)
graph = sns.catplot(x="model", y="vals", hue='cols', data=dfm_select, kind='point', legend_out=False, height=10, aspect=2, linewidth=1.5)
# sns.pointplot(x="model", y="vals", hue='cols', data=dfm_select)
plt.xlabel("Model No")
plt.xticks(rotation=45)
plt.ylabel("Adet")
plt.title("Model Entegration Satış Rakamları")
plt.legend(title='Entegration', loc='upper right');

In [ ]:
df_sales_select_top = df_sales_name_quantity_entegration_drop.head(20)
df_sales_select_top

In [ ]:
df_sales_select_top.loc[:,"product_name"] + " | " + df_sales_select_top.loc[:,"model"]

In [ ]:
fig, ax = plt.subplots(figsize=(16,9), dpi=100)
graph = sns.barplot(data=df_sales_select_top, x="toplam_satış_adet", y="product_name", color="brown")
# ax.set_xticks(np.arange(0, 101, 5))
plt.xlabel("Adet")
# plt.xticks(rotation=45)

# ax.set_yticklabels(df_sales_select_top.loc[:,"product_name"] + " " + df_sales_select_top.loc[:,"model"])
ax.set_yticks(range(len(df_sales_select_top)))  # Etiketlerin konumlarını ayarla
ax.set_yticklabels(list(df_sales_select_top.loc[:,"model"] + " | " + df_sales_select_top.loc[:,"product_name"]))

plt.ylabel("Ürün")
plt.title("Ürün Toplam Satış Top 20")

# for value on graph 1 way
graph.bar_label(ax.containers[0]);

# # for value on graph 2 way
# i=0
# for p in graph.patches:
#     height = p.get_height()
#     graph.text(p.get_x()+p.get_width()/2., height + 0.5,
#         df_seller_evaluation['olumsuz etkiler'].value_counts()[i],ha="center")
#     i += 1;

In [ ]:
df_sales_select_last = df_sales_name_quantity_entegration_drop.tail(20)
df_sales_select_last

In [ ]:
fig, ax = plt.subplots(figsize=(16,9), dpi=100)
graph = sns.barplot(data=df_sales_select_last, x="toplam_satış_adet", y="product_name", color="brown")
# ax.set_xticks(np.arange(0, 101, 5))
plt.xlabel("Adet")
# plt.xticks(rotation=45)

ax.set_yticks(range(len(df_sales_select_last)))  # Etiketlerin konumlarını ayarla
ax.set_yticklabels(list(df_sales_select_last.loc[:,"model"] + " | " + df_sales_select_last.loc[:,"product_name"]))

plt.ylabel("Ürün")
plt.title("Ürün Toplam Satış Last 20")

# for value on graph 1 way
graph.bar_label(ax.containers[0]);

##### Entegration Sales Number

In [ ]:
dfm

In [ ]:
dfm2 = dfm[dfm["cols"].isin(entegration_list_all)]
dfm2 = dfm2.dropna()
dfm2 = dfm2[dfm2["vals"] != 0]
dfm2.reset_index(drop=True, inplace=True)
dfm2

In [ ]:
plt.figure(figsize=(16,9),dpi=100)
graph = sns.countplot(x="cols", data=dfm2, color="green")
plt.xlabel("Entegration")
plt.xticks(rotation=45)
plt.ylabel("Adet")
plt.title("Entegration Tekil Ürün Satış")
# 1 way
# graph.bar_label(graph.containers[0])
# 2 way
for container in graph.containers:
    graph.bar_label(container)

# Note: Grafik pazaryerlerinde satılan ürün çeşidi sayısını göstermekte.

In [ ]:
dfm3 = dfm2.groupby(["cols"])[["vals"]].sum().reset_index()
dfm3

In [ ]:
dfm3.idxmax()

In [ ]:
dfm3.index.max()

In [ ]:
dfm3.loc[dfm3.index.max()+1,:] = {"cols":"Total", "vals":dfm3["vals"].sum()}  # Total sales quantity
dfm3

In [ ]:
# i = 0
# for value in values:
#     df.loc[df.index.max()+i,:] = {"cols":"Total", "vals":value}
#     i += 1 

In [ ]:
plt.figure(figsize=(16,9),dpi=100)
graph = sns.barplot(x="cols", y="vals", data=dfm3, color="orange",)
plt.xlabel("Entegration")
plt.xticks(rotation=45)
plt.ylabel("Adet")
plt.title("Entegration Toplam Satış")
# 1 way
graph.bar_label(graph.containers[0]);

# Note: Grafik pazaryerlerinde ve toplamda satılan toplam ürün sayısını (bir şipariş birden fazla ürün içerebileceğinden ürün sayısı olarak ele alındı) göstermekte.

##### Cancel Order Quantity

In [ ]:
df_entegra_select["status_name"].unique()

In [ ]:
df_entegra_select_cancel = df_entegra_select[(df_entegra_select["status_name"] == "İade-İptal")]
df_entegra_select_cancel.reset_index(drop=True, inplace=True)
df_entegra_select_cancel

In [ ]:
entegration_list_all = list(df_entegra_select["entegration"].unique())
entegration_list_all

In [ ]:
entegration_list_cancel = list(df_entegra_select_cancel["entegration"].unique())
entegration_list_cancel

In [ ]:
df_cancel_quantity_entegration = pd.pivot_table(index="model",columns="entegration", values="total_product_quantity", aggfunc='sum', data=df_entegra_select_cancel)
df_cancel_quantity_entegration = df_cancel_quantity_entegration.fillna(0)
df_cancel_quantity_entegration.reset_index(inplace=True)
df_cancel_quantity_entegration

In [ ]:
df_cancel_quantity_entegration = df_cancel_quantity_entegration.rename_axis(None, axis=1)  # del index name
df_cancel_quantity_entegration

In [ ]:
for entegration in entegration_list_all:
    if entegration not in entegration_list_cancel:
        df_cancel_quantity_entegration[f"{entegration}"] = 0
    else:
        pass
    
df_cancel_quantity_entegration

In [ ]:
# 1 way
# df_cancel_quantity_entegration["total_product_cancel"] = 0
# for entg_column in entegration_list_all:
#     df_cancel_quantity_entegration["total_product_cancel"] += df_cancel_quantity_entegration[f"{entg_column}"]

# 2 way
df_cancel_quantity_entegration["toplam_iptal_adet"] = df_cancel_quantity_entegration[entegration_list_all].sum(axis=1)

df_cancel_quantity_entegration

In [ ]:
# df_cancel_quantity_entegration["total_product_cancel"] = df_cancel_quantity_entegration["Amazon"] + df_cancel_quantity_entegration["Ciceksepeti"] + \
#     df_cancel_quantity_entegration["Hepsiburada"] + df_cancel_quantity_entegration["N11"] + df_cancel_quantity_entegration["Trendyol"] + df_cancel_quantity_entegration["Woocommerce"]
# df_cancel_quantity_entegration

In [ ]:
df_cancel_name_quantity_entegration  = pd.merge(df_cancel_quantity_entegration, df_entegra_select_product, how="left", on="model")
df_cancel_name_quantity_entegration.drop_duplicates(inplace=True)
df_cancel_name_quantity_entegration

In [ ]:
# df_cancel_name_quantity_entegration[[df_cancel_name_quantity_entegration.columns[-1]]]

In [ ]:
cols = list(df_cancel_name_quantity_entegration.columns)
cols = [cols[-1]] + cols[:-1]
df_cancel_name_quantity_entegration = df_cancel_name_quantity_entegration[cols]
df_cancel_name_quantity_entegration

In [ ]:
df_cancel_name_quantity_entegration = df_cancel_name_quantity_entegration.sort_values(by="toplam_iptal_adet", ascending=False)
df_cancel_name_quantity_entegration.reset_index(drop=True, inplace=True)
df_cancel_name_quantity_entegration

In [ ]:
df_cancel_name_quantity_entegration_drop = df_cancel_name_quantity_entegration.drop_duplicates(subset="model")
df_cancel_name_quantity_entegration_drop.reset_index(drop=True, inplace=True)
df_cancel_name_quantity_entegration_drop

In [ ]:
df_cancel_name_quantity_entegration_drop.to_excel(fr"{output_path}\Entegra Ürün İptal Rakamları Adet.xlsx", index=False)

##### Cancel Order TL

In [ ]:
df_entegra_select_cancel

In [ ]:
df_entegration_model_cancel = df_entegra_select_cancel.groupby(["model","entegration"])[["total","tax","grand_total"]].sum()
df_entegration_model_cancel = df_entegration_model_cancel.reset_index()
df_entegration_model_cancel.sort_values(by=["entegration","total"], ascending=False, inplace=True)
df_entegration_model_cancel.reset_index(drop=True, inplace=True)
df_entegration_model_cancel

###### İptal Satış

In [ ]:
df_cancel_entegration_total = pd.pivot_table(index="model",columns="entegration", values="total", aggfunc='sum', data=df_entegration_model_cancel)
df_cancel_entegration_total = df_cancel_entegration_total.fillna(0)
df_cancel_entegration_total.reset_index(inplace=True)
df_cancel_entegration_total = df_cancel_entegration_total.rename_axis(None, axis=1)
df_cancel_entegration_total

In [ ]:
for entegration in entegration_list_all:
    if entegration not in entegration_list_cancel:
        df_cancel_entegration_total[f"{entegration}"] = 0
    else:
        pass
    
df_cancel_entegration_total

In [ ]:
# 1 way
df_cancel_entegration_total["toplam_iptal_satış_tl"] = 0
for entg_column in entegration_list_all:
    df_cancel_entegration_total["toplam_iptal_satış_tl"] += df_cancel_entegration_total[f"{entg_column}"]

df_cancel_entegration_total 

In [ ]:
df_cancel_entegration_total = df_cancel_entegration_total.sort_values(by="toplam_iptal_satış_tl", ascending=False)
df_cancel_entegration_total.reset_index(drop=True, inplace=True)
df_cancel_entegration_total = df_cancel_entegration_total[["model","toplam_iptal_satış_tl"]]
df_cancel_entegration_total

###### İptal Vergi

In [ ]:
df_cancel_entegration_tax = pd.pivot_table(index="model",columns="entegration", values="tax", aggfunc='sum', data=df_entegration_model_cancel)
df_cancel_entegration_tax = df_cancel_entegration_tax.fillna(0)
df_cancel_entegration_tax.reset_index(inplace=True)
df_cancel_entegration_tax = df_cancel_entegration_tax.rename_axis(None, axis=1)
df_cancel_entegration_tax

In [ ]:
for entegration in entegration_list_all:
    if entegration not in entegration_list_cancel:
        df_cancel_entegration_tax[f"{entegration}"] = 0
    else:
        pass
    
df_cancel_entegration_tax

In [ ]:
# 1 way
df_cancel_entegration_tax["toplam_iptal_vergi_tl"] = 0
for entg_column in entegration_list_all:
    df_cancel_entegration_tax["toplam_iptal_vergi_tl"] += df_cancel_entegration_tax[f"{entg_column}"]

df_cancel_entegration_tax

In [ ]:
df_cancel_entegration_tax = df_cancel_entegration_tax.sort_values(by="toplam_iptal_vergi_tl", ascending=False)
df_cancel_entegration_tax.reset_index(drop=True, inplace=True)
df_cancel_entegration_tax = df_cancel_entegration_tax[["model","toplam_iptal_vergi_tl"]]
df_cancel_entegration_tax

###### Brüt İptal

In [ ]:
df_cancel_entegration_grand_total = pd.pivot_table(index="model",columns="entegration", values="grand_total", aggfunc='sum', data=df_entegration_model_cancel)
df_cancel_entegration_grand_total = df_cancel_entegration_grand_total.fillna(0)
df_cancel_entegration_grand_total.reset_index(inplace=True)
df_cancel_entegration_grand_total = df_cancel_entegration_grand_total.rename_axis(None, axis=1)
df_cancel_entegration_grand_total

In [ ]:
for entegration in entegration_list_all:
    if entegration not in entegration_list_cancel:
        df_cancel_entegration_grand_total[f"{entegration}"] = 0
    else:
        pass
    
df_cancel_entegration_grand_total

In [ ]:
# 1 way
df_cancel_entegration_grand_total["toplam_iptal_brüt_satış_tl"] = 0
for entg_column in entegration_list_all:
    df_cancel_entegration_grand_total["toplam_iptal_brüt_satış_tl"] += df_cancel_entegration_grand_total[f"{entg_column}"]

df_cancel_entegration_grand_total

In [ ]:
df_cancel_entegration_grand_total = df_cancel_entegration_grand_total.sort_values(by="toplam_iptal_brüt_satış_tl", ascending=False)
df_cancel_entegration_grand_total.reset_index(drop=True, inplace=True)
df_cancel_entegration_grand_total = df_cancel_entegration_grand_total[["model","toplam_iptal_brüt_satış_tl"]]
df_cancel_entegration_grand_total

In [ ]:
data_cancel_frames = [df_cancel_entegration_total, df_cancel_entegration_tax, df_cancel_entegration_grand_total]

In [ ]:
df_cancel_merged = reduce(lambda  left,right: pd.merge(left,right,on=['model'], how='inner'), data_cancel_frames)
df_cancel_merged = df_cancel_merged.drop_duplicates()
df_cancel_merged = df_cancel_merged.reset_index(drop=True)
df_cancel_merged

In [ ]:
df_quantity_cancel_entegration_merge  = pd.merge(df_cancel_name_quantity_entegration_drop, df_cancel_merged, how="left", on="model")
df_quantity_cancel_entegration_merge.drop_duplicates(inplace=True)
df_quantity_cancel_entegration_merge

In [ ]:
df_quantity_cancel_entegration_merge.to_excel(fr"{output_path}\Entegra Ürün İptal Satış Adet Ve TL.xlsx", index=False)

###### Cancel Duplicated

In [ ]:
df_cancel_dublicated_entegration = df_cancel_name_quantity_entegration[df_cancel_name_quantity_entegration.duplicated(subset="model", keep=False)]
df_cancel_dublicated_entegration

In [ ]:
df_cancel_dublicated_entegration.to_excel(fr"{output_path}\Entegra Ürün İptal Rakamları Adet Yenile.xlsx", index=False)

##### Cancel Visualization

In [ ]:
df_cancel_visual = df_cancel_name_quantity_entegration_drop.head(50)
df_cancel_visual

In [ ]:
select_cancel_visual_list = list(df_cancel_visual["model"])
# select_cancel_visual_list

In [ ]:
dfm2 = df_cancel_name_quantity_entegration_drop.melt(id_vars=["product_name","model"],var_name='cols', value_name='vals')
dfm2

In [ ]:
dfm_select2 = dfm2[dfm2["model"].isin(select_cancel_visual_list)]
dfm_select2

In [ ]:
plt.figure(figsize=(16,9),dpi=100)
graph = sns.catplot(x="model", y="vals", hue='cols', data=dfm_select2, kind='point', legend_out=False, height=10, aspect=2, linewidth=1.5)
# sns.pointplot(x="model", y="vals", hue='cols', data=dfm_select)
plt.xlabel("Model No")
plt.xticks(rotation=45)
plt.ylabel("Adet")
plt.title("Model İptal Rakamları")
plt.legend(title='Entegration', loc='upper right');

In [ ]:
df_cancel_select_top = df_cancel_name_quantity_entegration_drop.head(20)
df_cancel_select_top

In [ ]:
fig, ax = plt.subplots(figsize=(16,9), dpi=100)
graph = sns.barplot(data=df_cancel_select_top, x="toplam_iptal_adet", y="product_name", color="red")
# ax.set_xticks(np.arange(0, 101, 5))
plt.xlabel("Adet")
# plt.xticks(rotation=45)

# y eksenindeki etiketlere ekleme yapıyoruz. 
ax.set_yticks(range(len(df_cancel_select_top)))  # Etiketlerin konumlarını ayarla
ax.set_yticklabels(list(df_cancel_select_top.loc[:,"model"] + " | " + df_cancel_select_top.loc[:,"product_name"]))

plt.ylabel("Ürün")
plt.title("Ürün Toplam İptal Top 20")

# for value on graph 1 way
graph.bar_label(ax.containers[0]);

In [ ]:
df_cancel_select_last = df_cancel_name_quantity_entegration_drop.tail(20)
df_cancel_select_last

In [ ]:
fig, ax = plt.subplots(figsize=(16,9), dpi=100)
graph = sns.barplot(data=df_cancel_select_last, x="toplam_iptal_adet", y="product_name", color="red")
# ax.set_xticks(np.arange(0, 101, 5))
plt.xlabel("Adet")
# plt.xticks(rotation=45)
# y eksenindeki etiketlere ekleme yapıyoruz. 
ax.set_yticks(range(len(df_cancel_select_last)))  # Etiketlerin konumlarını ayarla
ax.set_yticklabels(list(df_cancel_select_last.loc[:,"model"] + " | " + df_cancel_select_last.loc[:,"product_name"]))

plt.ylabel("Ürün")
plt.title("Ürün Toplam Satış Last 20")

# for value on graph 1 way
graph.bar_label(ax.containers[0]);

##### Add Cancel Order To Sales

###### Part 1

In [ ]:
df_sales_name_quantity_entegration_drop

In [ ]:
df_cancel_name_quantity_entegration_drop_var = df_cancel_name_quantity_entegration_drop[["model","toplam_iptal_adet"]]
df_cancel_name_quantity_entegration_drop_var

In [ ]:
df_sales_cancel_quantity_all = pd.merge(df_sales_name_quantity_entegration_drop, df_cancel_name_quantity_entegration_drop_var, how="left", on="model")
df_sales_cancel_quantity_all.fillna(0, inplace=True)
df_sales_cancel_quantity_all

In [ ]:
df_sales_cancel_quantity_all.to_excel(fr"{output_path}\Entegra Ürün Satış Ve İptal Rakamları Adet Master.xlsx", index=False)

###### Part 2

In [ ]:
df_sales_merged

In [ ]:
df_cancel_merged

In [ ]:
df_sales_cancel_quantity_tl_all = pd.merge(df_sales_cancel_quantity_all, df_sales_merged, how="left", on="model")
df_sales_cancel_quantity_tl_all2 = pd.merge(df_sales_cancel_quantity_tl_all, df_cancel_merged, how="left", on="model")
df_sales_cancel_quantity_tl_all2.fillna(0, inplace=True)
df_sales_cancel_quantity_tl_all2

In [ ]:
df_sales_cancel_quantity_tl_all2.to_excel(fr"{output_path}\Entegra Ürün Satış Ve İptal Adet Ve TL Master.xlsx", index=False)

##### Sales Cancel Visualization

In [ ]:
df_sales_cancel_all_select = df_sales_cancel_quantity_all.head(30)
df_sales_cancel_all_select

In [ ]:
fig = plt.figure(figsize=(16,9), dpi=100)
ax = fig.add_axes([0,0,1,1])
ax.plot(df_sales_cancel_all_select["model"],df_sales_cancel_all_select["toplam_satış_adet"],label='Ürün Satış')
ax.plot(df_sales_cancel_all_select["model"],df_sales_cancel_all_select["toplam_iptal_adet"],label="Ürün İptal")
ax.set_title("Ürün Satış Ve İptal")
ax.set_ylabel("Miktar Adet")
ax.set_xlabel("Ürün")
plt.xticks(rotation=45)
#ax.set_xticklabels(ax.get_xticks(), rotation = 40)
ax.legend(loc=0);

In [ ]:
df_sales_total_tax = pd.DataFrame(df_entegra_select_sales[["total","tax","grand_total"]].sum()).reset_index()
df_sales_total_tax.rename(columns={"index":"name", 0:"value"}, inplace=True)
df_sales_total_tax

In [ ]:
fig, ax = plt.subplots(figsize=(16,9), dpi=100)
graph = sns.barplot(data=df_sales_total_tax, x="name", y="value", color="green")
# ax.set_xticks(np.arange(0, 101, 5))
plt.xlabel("Ad")
# plt.xticks(rotation=45)

# x eksenindeki etiketleri büyük harf yapalım
xticklabel_var = [label.get_text().capitalize() for label in graph.get_xticklabels()]
ax.set_xticks(range(len(xticklabel_var))) # Etiketlerin konumlarını ayarla
ax.set_xticklabels(xticklabel_var)


plt.ylabel("Miktar TL")
plt.title("Grand Total-Tax-Sales")

# for value on graph 1 way
graph.bar_label(ax.containers[0]);

In [ ]:
df_cancel_total_tax = pd.DataFrame(df_entegra_select_cancel[["total","tax","grand_total"]].sum()).reset_index()
df_cancel_total_tax.rename(columns={"index":"name", 0:"value"}, inplace=True)
df_cancel_total_tax

In [ ]:
fig, ax = plt.subplots(figsize=(16,9), dpi=100)
graph = sns.barplot(data=df_cancel_total_tax, x="name", y="value", color="red")
# ax.set_xticks(np.arange(0, 101, 5))
plt.xlabel("Ad")
# plt.xticks(rotation=45)

# x eksenindeki etiketleri büyük harf yapalım
xticklabel_var = [label.get_text().capitalize() for label in graph.get_xticklabels()]
ax.set_xticks(range(len(xticklabel_var))) # Etiketlerin konumlarını ayarla
ax.set_xticklabels(xticklabel_var)


plt.ylabel("Miktar TL")
plt.title("Grand Total-Tax-Cancel")

# for value on graph 1 way
graph.bar_label(ax.containers[0]);

In [ ]:
# for i in graph.get_xticklabels():
#     print(i.get_text())

In [ ]:
# for i in [label.get_text() for label in graph.get_xticklabels()]:
#     print(str.capitalize(i))

In [ ]:
df_entegra_total_tax = df_entegra_select[["entegration","total","tax","grand_total"]]
df_entegra_total_tax

In [ ]:
df_grouped = df_entegra_total_tax.groupby('entegration').sum().reset_index()

# Melt işlemi ile total, tax, grand_total sütunlarını birleştirelim
df_melted = pd.melt(df_grouped, id_vars=['entegration'], value_vars=['total', 'tax', 'grand_total'], var_name='value_type', value_name='value')

# Grafik çizimi için seaborn barplot kullanalım
plt.figure(figsize=(16, 9))
graph = sns.barplot(data=df_melted, x='entegration', y='value', hue='value_type', palette='bright')

# Grafik başlığı ve eksen etiketlerini ekleyelim
plt.title('Toplam, Vergi Ve Grand Total')
plt.xlabel('Entegrasyon')
plt.ylabel('Miktar TL')

# Görseli gösterelim
plt.tight_layout()
plt.show()

In [ ]:
df_grouped = df_entegra_total_tax.groupby('entegration').sum().reset_index()

# Melt işlemi ile total, tax, grand_total sütunlarını birleştirelim
df_melted = pd.melt(df_grouped, id_vars=['entegration'], value_vars=['total', 'tax', 'grand_total'], var_name='value_type', value_name='value')

# Grafik çizimi için seaborn barplot kullanalım
plt.figure(figsize=(16, 9))
graph = sns.barplot(data=df_melted, x='value_type', y='value', hue='entegration', palette='bright')

# Grafik başlığı ve eksen etiketlerini ekleyelim
plt.title('Toplam, Vergi Ve Grand Total')
plt.xlabel('Value Type')
plt.ylabel('Miktar TL')

# Görseli gösterelim
plt.tight_layout()
plt.show()

##### Sales Time Analysis

In [ ]:
df_entegra_select_sales

In [ ]:
df_sales_time_select = df_entegra_select_sales[["date","time","total_product_quantity","total","tax","grand_total"]]
df_sales_time_select.head(5)

In [ ]:
df_sales_time_select.loc[:,"month"] = pd.to_datetime(df_sales_time_select.loc[:,"date"], format='%Y-%m-%d').dt.month
df_sales_time_select.loc[:,"hour"] = pd.to_datetime(df_sales_time_select.loc[:,"time"], format='%H:%M:%S').dt.hour
df_sales_time_select.head(5)

In [ ]:
df_sales_time_month = df_sales_time_select.groupby("month")[["total_product_quantity","total","tax","grand_total"]].sum().reset_index()
df_sales_time_month

In [ ]:
fig, ax = plt.subplots(figsize=(16,9), dpi=100)

# graph = sns.barplot(data=df_entegra_select_sales, x=pd.to_datetime(df_entegra_select_sales["date"]).dt.month, y='total', estimator="sum")
graph = sns.barplot(data=df_sales_time_month, x='month', y='total', color='green')

# Grafik başlığı ve eksen etiketlerini ekleyelim
plt.title('Aylık Toplam Satış')
plt.xlabel('Ay')
plt.ylabel('Miktar TL')
graph.bar_label(ax.containers[0])

# Görseli gösterelim
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9), dpi=100)

# graph = sns.barplot(data=df_entegra_select_sales, x=pd.to_datetime(df_entegra_select_sales["date"]).dt.month, y='total', estimator="sum")
graph = sns.barplot(data=df_sales_time_month, x='month', y='tax', color='red')

# Grafik başlığı ve eksen etiketlerini ekleyelim
plt.title('Aylık Satış Vergi')
plt.xlabel('Ay')
plt.ylabel('Miktar TL')
graph.bar_label(ax.containers[0])

# Görseli gösterelim
plt.tight_layout()
plt.show()

In [ ]:
df_sales_time_hour = df_sales_time_select.groupby("hour")[["total","tax","grand_total"]].sum().reset_index()
df_sales_time_hour

In [ ]:
fig, ax = plt.subplots(figsize=(16,9), dpi=100)

# graph = sns.barplot(data=df_entegra_select_sales, x=pd.to_datetime(df_entegra_select_sales["date"]).dt.month, y='total', estimator="sum")
graph = sns.pointplot(data=df_sales_time_hour, x='hour', y='total', color='green')

# Grafik başlığı ve eksen etiketlerini ekleyelim
plt.title('Saatlik Toplam Satış')
plt.xlabel('Hour')
plt.ylabel('Miktar TL')

# Görseli gösterelim
plt.tight_layout()
plt.show()